# Tutorial

In [1]:
import os
import sys
import logging
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import hurraypy as hr
import numpy as np

In [3]:
hr.__version__

'0.0.3'

First, make sure all logging messages are sent to stdout:

In [4]:
logger = logging.getLogger('hurraypy')

console = logging.StreamHandler()
console.setLevel(logging.DEBUG)
console.setFormatter(logging.Formatter('%(levelname)s --- %(message)s'))
logger.addHandler(console)
logger.setLevel(logging.DEBUG)

In [5]:
logger.handlers

In [6]:
hr.log.log.debug("bla")
hr.log.log.info("bla")

DEBUG --- bla
INFO --- bla


Make sure a hurray server is running!

In [7]:
conn = hr.connect('localhost', '2222')
print(conn)

<Connection (host=localhost, port=2222)>


## THIS FAILS!

In [8]:
f2 = conn.create_file("test_asdf.h5", overwrite=True)

f2.delete()

f2 = conn.create_file("test_asdfXXXX.h5", overwrite=True)

f2 = conn.create_file("test_asdf.h5", overwrite=True)

DEBUG --- Sending 54 bytes...
DEBUG --- Handle request (Protocol: v1, Msg size: 9)
DEBUG --- Read total of 17 bytes ...
DEBUG --- Sending 43 bytes...
DEBUG --- Handle request (Protocol: v1, Msg size: 9)
DEBUG --- Read total of 17 bytes ...
DEBUG --- Sending 58 bytes...
DEBUG --- Handle request (Protocol: v1, Msg size: 9)
DEBUG --- Read total of 17 bytes ...
DEBUG --- Sending 54 bytes...


error: unpack requires a bytes object of length 4

In [ ]:
f = conn.create_file("test.h5", overwrite=True)

In [ ]:
f

In [ ]:
conn.File

In [ ]:
f = conn.File("test.h5")

In [ ]:
f

In [ ]:
with conn.File("test.h5") as f:
    print(f)

In [ ]:
f.attrs["bla"] = 123

In [ ]:
f.attrs["bla"]

Objects/Nodes containing attributes get a red "A":

In [ ]:
f

In [ ]:
f.create_group("test")

In [ ]:
f.keys()

In [ ]:
f["test"]

In [ ]:
grp = f.create_group("mygrp/data")

In [ ]:
f.keys()

In [ ]:
f["mygrp/data"]

In [ ]:
grp.path

In [ ]:
f["/mygrp"].attrs["foo"] = "bar"

In [ ]:
f["/mygrp"]

In [ ]:
data = np.random.random((600, 400))

In [ ]:
grp.create_dataset("myarray", data=data)

Creating a dataset has increased filesize:

In [ ]:
f

In [ ]:
dst = f["/mygrp/data/myarray"]

In [ ]:
dst

In [ ]:
dst.attrs["unit"] = "celsius"

In [ ]:
dst

In [ ]:
dst.shape

In [ ]:
dst.dtype

The following reads the whole dataset into a NumPy array:

In [ ]:
dst[:]

Fancy indexing allows to read/write only portions of a dataset:

In [ ]:
dst[0, 0:3] = 999

In [ ]:
dst[0, 0:4]

## Exploring file structures

Every group has a ``tree()`` method that displays sub groups and datasets as a tree.

In [ ]:
f.require_group("/mygrp/data/somegroup")
f.require_group("/mygrp/datagrp2")
f.require_group("/mygrp/datagrp3")
f.require_group("/test/test2/test3")

In [ ]:
f.tree()

In [ ]:
f["/mygrp/data"].tree()

If you're not in a notebook or ipython console, ``tree()`` will give you a text based representation:

In [ ]:
print(f.tree())

## Renaming and deleting files

In [ ]:
f2 = conn.create_file("test_asdf.h5", overwrite=True)

f2.delete()

f2 = conn.create_file("test_asdfXXXX.h5", overwrite=True)

f2 = conn.create_file("test_asdf.h5", overwrite=True)

In [ ]:
f2 = f2.rename("test_asdff.h5")

In [ ]:
f2

In [ ]:
conn.File("test_asdff.h5")

Creating files outside base directory does not work:

In [ ]:
conn.create_file("/home/rg/Desktop/test.h5", overwrite=True)